<a href="https://colab.research.google.com/github/LejoFlores/EarthScienceResearchComputing/blob/master/ClipAndExport_SRP_landuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=oLSsmxbR_eisDNh8SQsobwJkpceESc-MtgHyEnlHzcc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/5AGIZw4KObDOIi7Z9xijLVMqrsa3VTFXEnoRyRnoDTXB907ScjzYPiU

Successfully saved authorization token.


In [ ]:

# Algorithm:
# 1. Get counties of interest in Snake River Plain (see table below)
# 2. Create a new feature with selected counties
# 3. Union the counties, export the counties
# 4. Get the NLCD data
# 5. Clip to the area of the county
# 6. Export 

#--------------------------------#
#   County         |  FIPS code  #
#--------------------------------#
#   Ada            |   16001     #
#   Bannock        |   16005     #
#   Bingham        |   16011     #
#   Blaine         |   16013     #
#   Bonneville     |   16019     #
#   Butte          |   16023     #
#   Camas          |   16025     #
#   Canyon         |   16027     #
#   Cassia         |   16031     #
#   Clark          |   16033     #
#   Elmore         |   16039     #
#   Fremont        |   16043     #
#   Gem            |   16045     #
#   Gooding        |   16047     #
#   Jefferson      |   16051     #
#   Jerome         |   16053     #
#   Lincoln        |   16063     #
#   Madison        |   16065     #
#   Malheur (OR)   |   41045     #
#   Minidoka       |   16067     #
#   Owyhee         |   16073     #
#   Payette        |   16075     #
#   Power          |   16077     #
#   Teton          |   16081     #
#   Twin Falls     |   16083     #
#   Washington     |   16087     #
#--------------------------------#



In [ ]:
counties = ee.FeatureCollection("TIGER/2016/Counties")


srb_countyfp_list = ee.List(['001','005','011','013','019','023','025', \
                             '027','031','033','039','043','045','047', \
                             '051','053','063','065','067','073','075', \
                             '077','081','083','087'])

id_srb_cnty = counties.filter(ee.Filter.eq('STATEFP','16')) \
                      .filter(ee.Filter.inList('COUNTYFP', srb_countyfp_list)) \
           
or_srb_cnty = counties.filter(ee.Filter.eq('STATEFP','41')) \
                   .filter(ee.Filter.eq('COUNTYFP','045'))

srb_cnty = id_srb_cnty.merge(or_srb_cnty)

srb_bnd = srb_cnty.union()

# Create a new map and add the watershed we will use to clip the EVI data
Map = emap.Map(center=(43.6150, -116.2023),zoom=6)

Map.addLayer(ee.Image().paint(srb_bnd, 0, 2), {}, 'Snake River Plain')
Map.addLayerControl()
Map

In [ ]:
dataset = ee.ImageCollection('USGS/NLCD').filterBounds(srb_bnd)
srb_landcover = dataset.select('landcover') \
                    .map(lambda image: image.clip(srb_bnd))

landcoverVis = {
  'min': 0.0,
  'max': 95.0,
  'palette': ['000000','000000','000000','000000','000000','000000','000000',
    '000000','000000','000000','000000','466b9f','d1def8','000000','000000',
    '000000','000000','000000','000000','000000','000000','dec5c5','d99282',
    'eb0000','ab0000','000000','000000','000000','000000','000000','000000',
    'b3ac9f','000000','000000','000000','000000','000000','000000','000000',
    '000000','000000','68ab5f','1c5f2c','b5c58f','000000','000000','000000',
    '000000','000000','000000','000000','af963c','ccb879','000000','000000',
    '000000','000000','000000','000000','000000','000000','000000','000000',
    '000000','000000','000000','000000','000000','000000','000000','000000',
    'dfdfc2','d1d182','a3cc51','82ba9e','000000','000000','000000','000000',
    '000000','000000','dcd939','ab6c28','000000','000000','000000','000000',
    '000000','000000','000000','b8d9eb','000000','000000','000000','000000',
    '6c9fb8']}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(srb_landcover, landcoverVis, 'Landcover')
Map


In [ ]:
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'srp-nlcd'}  # scale means resolution.
img_lst = srb_landcover.toList(100)

count = srb_landcover.size().getInfo()

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image.clip(srb_bnd.geometry()), name, downConfig)
    task.start()
    print(task.status())

NLCD1992
{'state': 'READY', 'description': 'NLCD1992', 'creation_timestamp_ms': 1602450996161, 'update_timestamp_ms': 1602450996161, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'D35E3TPVLPHPIXST7VZDLMRD', 'name': 'projects/earthengine-legacy/operations/D35E3TPVLPHPIXST7VZDLMRD'}
NLCD2001
{'state': 'READY', 'description': 'NLCD2001', 'creation_timestamp_ms': 1602451018960, 'update_timestamp_ms': 1602451018960, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UNT7DBP4SQRBURSUVD466POV', 'name': 'projects/earthengine-legacy/operations/UNT7DBP4SQRBURSUVD466POV'}
NLCD2004
{'state': 'READY', 'description': 'NLCD2004', 'creation_timestamp_ms': 1602451040720, 'update_timestamp_ms': 1602451040720, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'MXGCVL6NHIIBKYX62RCAY43D', 'name': 'projects/earthengine-legacy/operations/MXGCVL6NHIIBKYX62RCAY43D'}
NLCD2006
{'state': 'READY', 'description': 'NLCD2006', 'creation_timestamp_ms': 1602451059662, 'update_timestamp

In [ ]:
task.status()

{'creation_timestamp_ms': 1602451143507,
 'description': 'NLCD2016',
 'id': 'VHMYVOWJI5VC7UR4CGUYWXAS',
 'name': 'projects/earthengine-legacy/operations/VHMYVOWJI5VC7UR4CGUYWXAS',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1602451143507}